## (Scrapping)Mouad DAHDOUH
## Site: https://www.mubawab.ma # Ville : Tanger


In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import re
import pandas as pd
import numpy as np

def Get_detail_mubawab_ma(URL):
    
    html = urlopen(URL)
    bs_ = BeautifulSoup(html,'html.parser')
    title = bs_.find('h1',{'class':'searchTitle'}).get_text().strip()
    try:
        price = bs_.find('h3',{'class':'orangeTit'}).get_text().strip()
        
    except AttributeError:
        price=np.nan
        
    localisation = bs_.find('h3',{'class':'greyTit'}).get_text().strip()
    size,piece,nb_chambre,salles_de_bains,etat,old,etage,Caracteristiques_supplementaires = '','','','','','','',''
    try:
        desc = bs_.findAll('span',{'class':'tagProp'})
        
        if desc[0].get_text().split()[1] == "m²":
            size=desc[0].get_text().strip()
        else :
            size = np.nan
            
        if desc[1].get_text().split()[1] == "Pièces" or desc[1].get_text().split()[1] == "Pièce":
            piece=desc[1].get_text()
        else:
            piece= np.nan
            
        if desc[2].get_text().split()[1] == "Chambres" or desc[2].get_text().split()[1] == "Chambre" :
            nb_chambre=desc[2].get_text()
        else:
            nb_chambre=np.nan
            
        if desc[3].get_text().split()[2] == "de":
            salles_de_bains= desc[3].get_text()
        else:
            salles_de_bains= np.nan
            
        etat = desc[4].get_text().strip()
        
        liste = desc[5].get_text().split()
 
        for word in liste:
            if word == "ans":
                old = desc[5].get_text().strip() 
                break
            else:
                old = np.nan
                
        
          
        if desc[6].get_text().split()[1] == "étage":
            etage = desc[6].get_text().strip() 
        else:
            etage = np.nan
        
    except IndexError:
        pass
        
    features = bs_.findAll('li',{'class':'col-3 floatR fSize14'})
    Caracteristiques_supplementaires =""
    for a in features:
        Caracteristiques_supplementaires = Caracteristiques_supplementaires + "|" +a.get_text().strip()
   

    return title,price,localisation,size,piece,nb_chambre,salles_de_bains,etat,old,etage,Caracteristiques_supplementaires,URL

def Get_mubawab_ma(PageNumber):
    links = []
    #html = urlopen("https://www.mubawab.ma/fr/ct/tanger/immobilier-a-vendre-all:sc:apartments-for-sale,commercial-property-for-sale,farms-for-sale,houses-for-sale,land-for-sale,offices-for-sale,riads-for-sale,villas-and-luxury-homes-for-sale:p:{}".format(PageNumber))
    html = urlopen("https://www.mubawab.ma/fr/ct/tanger/immobilier-a-louer-all:sc:apartments-for-rent,rooms-for-rent:p:{}".format(PageNumber))
    bs = BeautifulSoup(html,'html.parser')
    links_SpremiumBox_w100 = bs.findAll('li',{'class':'SpremiumBox w100'})
    links_premiumBox_w100 = bs.findAll('li',{'class':'premiumBox w100'})
    links_basicList = bs.findAll('li',{'class':'basicList'})
    
    for link in links_SpremiumBox_w100: 
        links.append(link.attrs['linkref'])
    for link in links_premiumBox_w100:
        links.append(link.attrs['linkref'])
    for link in links_basicList:
        links.append(link.attrs['linkref'])
     
    data = {'Titre': [],'Price':[],'Localisation':[],'Size':[],'Nb_pieces':[],'Nb_chambre':[],'Nb_Salles_bain':[]
          ,'Etat':[],'Old':[],'Etage':[],'Caracteristiques_supplementaires':[],"URL":[]}
            

        
    for link in links:
        Titre,Price,Localisation,Size,Nb_pieces,Nb_chambre,Nb_Salles_bain,Etat,Old,Etage,Caracteristiques_supplementaires,URL=Get_detail_mubawab_ma(link)
        data['Titre'].append(Titre)
        data['Price'].append(Price)
        data['Localisation'].append(Localisation)
        data['Size'].append(Size)
        data['Nb_pieces'].append(Nb_pieces)
        data['Nb_chambre'].append(Nb_chambre)
        data['Nb_Salles_bain'].append(Nb_Salles_bain)
        data['Etat'].append(Etat)
        data['Old'].append(Old)
        data['Etage'].append(Etage)
        data['Caracteristiques_supplementaires'].append(Caracteristiques_supplementaires)
        data['URL'].append(URL)
        
    return pd.DataFrame(data)
        
pages = ['1']
appended_data = []
for page in range(52) :#total 53
        
    data = Get_mubawab_ma(str(page))
    appended_data.append(data)
appended_data = pd.concat(appended_data)
 

In [3]:
appended_data.head(20)


,Titre,Price,Localisation,Size,Nb_pieces,Nb_chambre,Nb_Salles_bain,Etat,Old,Etage,Caracteristiques_supplementaires,URL
0,Appartement Meublé à Louer Place Mozart Tanger,10 000 DH,Mozart à\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\tTanger,125\n\t\t\t\t\tm²,4 Pièces,2 Chambres,2 Salles de bains,Nouveau,1-5 ans,6ème\n\t\t\t\t\tétage,|Meublé|Garage|Ascenseur|Concierge|Salon europ...,https://www.mubawab.ma/fr/a/6804355/appartemen...
1,APPART MEUBLE DE LUXE,4 500 DH,Moujahidine à\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\tT...,65\n\t\t\t\t\tm²,3 Pièces,2 Chambres,1 Salle de bain,Nouveau,NaN,1er\n\t\t\t\t\tétage,|Meublé|Ascenseur|Jardin|Concierge|Salon Maroc...,https://www.mubawab.ma/fr/a/6990528/appart-meu...
2,Appartement meublé à la location longue durée,13 000 DH,Malabata à\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\tTanger,124\n\t\t\t\t\tm²,5 Pièces,2 Chambres,2 Salles de bains,Bon état,NaN,NaN,|Meublé|Terrasse|Garage|Concierge|Vue sur mer|...,https://www.mubawab.ma/fr/a/6227401/appartemen...
3,Appartement Meublé à Louer – Place Mozart – Ta...,5 000 DH,Mozart à\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\tTanger,80\n\t\t\t\t\tm²,2 Pièces,2 Chambres,1 Salle de bain,Bon état,5-10 ans,4ème\n\t\t\t\t\tétage,|Meublé|Ascenseur|Concierge|Salon Marocain|Sal...,https://www.mubawab.ma/fr/a/7001236/appartemen...
4,Coquet Appartement à la location Malabata,11 500 DH,Malabata à\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\tTanger,120\n\t\t\t\t\tm²,4 Pièces,2 Chambres,,,,,|Meublé|Terrasse|Garage|Ascenseur,https://www.mubawab.ma/fr/a/6719792/coquet-app...
5,Location Appartement 130 m² MALABATA Tanger Re...,13 500 DH,Tanger,130\n\t\t\t\t\tm²,4 Pièces,3 Chambres,2 Salles de bains,130\n\t\t\t\t\tm²,NaN,NaN,|Meublé|Terrasse|Garage|Piscine|Jardin|Sécurité,https://www.mubawab.ma/fr/a/7005744/location-a...
6,Appart trés moderne et luxueux à vendre,2 300 000 DH,Centre à\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\tTanger,206\n\t\t\t\t\tm²,5 Pièces,3 Chambres,2 Salles de bains,Bon état,1-5 ans,NaN,|Terrasse|Ascenseur|Concierge|Cheminée|Cuisine...,https://www.mubawab.ma/fr/a/7005452/appart-tr%...
7,Appartement HAUT STANDING à louer,6 200 DH,Centre à\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\tTanger,125\n\t\t\t\t\tm²,4 Pièces,3 Chambres,2 Salles de bains,Nouveau,1-5 ans,NaN,|Terrasse|Garage|Ascenseur|Concierge|Sécurité|...,https://www.mubawab.ma/fr/a/6962123/appartemen...
8,Joli appartement meublé au Quartier Administratif,8 500 DH,Administratif à\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\...,90\n\t\t\t\t\tm²,4 Pièces,2 Chambres,1 Salle de bain,90\n\t\t\t\t\tm²,NaN,NaN,|Meublé,https://www.mubawab.ma/fr/a/5932757/joli-appar...
9,Appartement deux chambres à louer,4 000 DH,Centre à\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\tTanger,95\n\t\t\t\t\tm²,3 Pièces,2 Chambres,1 Salle de bain,Nouveau,NaN,8ème\n\t\t\t\t\tétage,|Ascenseur|Carrelage,https://www.mubawab.ma/fr/a/6990744/appartemen...


In [4]:
appended_data.to_excel('MubawabMa.xlsx', index=False, encoding='utf-8')

In [5]:
appended_data.to_csv('MubawabMa.csv', index=False, encoding='utf-8')   

In [6]:
appended_data.isnull().sum()

Titre                                 0
Price                                45
Localisation                          0
Size                                 95
Nb_pieces                           246
Nb_chambre                          235
Nb_Salles_bain                        0
Etat                                  0
Old                                 794
Etage                               782
Caracteristiques_supplementaires      0
URL                                   0
dtype: int64

In [7]:
appended_data.shape

(1683, 12)